In [ ]:
import os
import geopandas as gpd
import pandas as pd

# os.environ["USE_PYGEOS"] = os.environ.get("USE_PYGEOS", "0")

local_crs = 32636
# path to data
example_data_path = "../data"

In [ ]:
from masterplan_tools.method import BlocksCutter
from masterplan_tools.method.blocks import BlocksCutterGeometries

city_geometry = gpd.read_parquet(os.path.join(example_data_path, "city_geometry.parquet")).to_crs(local_crs)
water_geometry = gpd.read_parquet(os.path.join(example_data_path, "water_geometry.parquet")).to_crs(local_crs)
roads_geometry = gpd.read_parquet(os.path.join(example_data_path, "roads_geometry.parquet")).to_crs(local_crs)
railways_geometry = gpd.read_parquet(os.path.join(example_data_path, "railways_geometry.parquet")).to_crs(local_crs)
#ne nuzhna?????
nature_geometry = gpd.read_parquet(os.path.join(example_data_path, "nature_geometry_boundaries.parquet")).to_crs(local_crs)

In [ ]:
# load data required for blocks cutter
no_development = gpd.read_file(os.path.join(example_data_path, "no_development_pzz.geojson"), mask=city_geometry.to_crs(4326)).to_crs(local_crs)
no_development = no_development[no_development['RAYON']=='Василеостровский']
landuse = gpd.read_file(os.path.join(example_data_path, "landuse_zone_pzz.geojson"), mask=city_geometry.to_crs(4326)).to_crs(local_crs)
okn = gpd.read_file(os.path.join(example_data_path, "petropavlovsraya_delete.geojson"), mask=city_geometry.to_crs(4326)).to_crs(local_crs)

# no_development = pd.concat([no_development[["geometry"]], okn[["geometry"]]], axis=0)

In [ ]:
geoms = BlocksCutterGeometries(
  city=city_geometry,
  water=water_geometry,
  roads=roads_geometry,
  railways=railways_geometry,
  nature=nature_geometry, 
  no_dev=no_development,
  landuse=landuse
)

In [ ]:
from masterplan_tools.method.blocks import BlocksCutter

blocks = BlocksCutter(geometries = geoms).get_blocks() 

In [ ]:
blocks.to_gdf().explore()

In [ ]:
landuse.explore()

In [ ]:
no_development.explore()

In [ ]:
blocks.to_gdf().explore()

In [ ]:
from masterplan_tools.method import LuFilter

blocks = LuFilter(blocks.to_gdf(), landuse_geometries = geoms).filter_lu()

In [ ]:
"""
There are three landuse tags in the blocks gdf:
    1. 'no_dev_area' -- according to th no_debelopment gdf and cutoff without any buildings or specified / selected landuse types;
    2. 'selected_area' -- according to the landuse gdf. We separate theese polygons since they have specified landuse types;
    3. 'buildings' -- there are polygons that have buildings landuse type. 

    In further calculations we will use the in the following steps:
    Only 'buildings' -- to find clusters of buildings in big polygons;
    All of them while calculating the accessibility times among city blocks;
    All of them except 'no_dev_area' while optimizing the development of new facilities.

"""

In [ ]:
from masterplan_tools.method.blocks.blocks_clustering import BlocksClusterization

buildings_geom = gpd.read_file(os.path.join(example_data_path, "buildings_blocks.geojson"), mask=city_geometry.to_crs(4326)).to_crs(local_crs)
blocks = BlocksClusterization(blocks, buildings_geom).run()

In [ ]:
blocks[blocks["landuse"].isin(["new_buildings", "buildings"])].explore()

In [ ]:
blocks.explore()